**Introduction**

This is a Google Colab notebook in which you can run the deep learning system on default and custom DICOM files. The notebook consits of text cells (similar to this one) and code cells (similar to the next one). In order to run the code cells, you have to press the play button in the top left corner of the code cell.

**Step 0: Set Runtime type**

Colab provides GPUs (fast computational devices) to accelerate procesing. To enable it, go to *Runtime/Change runtime type* and select *GPU* as *Hardware accelerator*.


**Step 1: Imports**

First, run the following code cell to import every necessary python packages and preprocessing code. This may take a few seconds, you will see a green tick next to the play button if it is done.

In [ ]:
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch
from torchvision import models

!git clone https://github.com/mr-BM/RVEnet-demo.git
%cd RVEnet-demo

!pip install pydicom
!pip install planar
from preprocessing import get_preprocessed_frames

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

**Step 2: Upload DICOM file - Optional**

You can use a sample file to test the system, but in order to run the model on your DICOM file, first you have to upload it to Colab. You can do that by clicking on the folder icon in the left side of the window, and then on the upload icon. Uploading may take a few seconds or minutes (depending on your network connection).

**Step 3: Set parameters**

In this cell you can set the parameters of the sample/uploaded DICOM file as well as the link to the neural network model. Set every necessary value (marked by # required) in the next code cell, and run it.

In [ ]:
# REQUIRED
model_URL = "https://www.dropbox.com/s/d1w0nh1rzclo4ox/full_ensemble_model.pt?dl=1"

# OPTIONAL
DICOM_filename = "sample1.dcm"
# NOTE: If you want to run the system on your custom DICOM file, change this file name to the uploaded DICOM file's name, otherwise keep it "sample1.dcm".

# OPTIONAL
custom_FPS = None
# NOTE: FPS of the video data in the DICOM file. Set this value only if the DICOM does not contain it. In case you are not sure about this, try to run the code with the default None value and provide custom value if error occurs.
# example: 55

# OPTIONAL
pulse = None
# NOTE: Heart rate from the dicom. Set this value only if DICOM does not contain it. In case you are not sure about this, try to run the code with the default None value and provide custom value if error occurs.
# example: 55

# OPTIONAL
orientation = 'Mayo'
# NOTE: Use 'Mayo' or 'Stanford'


#### DOWNLOAD CODE ####
if not os.path.exists("/content/model.pt"):
    !wget {model_URL} -O /content/model.pt

if DICOM_filename=="sample1.dcm" and not os.path.exists("/content/sample1.dcm"):
    !wget https://www.dropbox.com/s/eqj3uhe1ckijn0y/sample1.dcm?dl=1 -O /content/sample1.dcm


**Step 4: Preprocessing**

By running the next code cell, the DICOM video will be extracted and preprocessed.

In [ ]:
preprocessed_frames = get_preprocessed_frames(os.path.join("/content", DICOM_filename), custom_FPS, pulse, orientation)

plt.figure()
plt.imshow(preprocessed_frames[0][0][2], cmap='gray') 
plt.show()

plt.figure()
plt.imshow(preprocessed_frames[0][0][0], cmap='gray') 
plt.show()

**Step 5: Run model on data**

As the final step, you can run the model on the uplaoded video data using the next code cell.

In [ ]:
preprocessed_frames = preprocessed_frames.to(device, dtype=torch.float)

model = torch.load("/content/model.pt")
model.eval()

heart_cycle_predictions = []

for heart_cycle in preprocessed_frames:
    output = model(heart_cycle)
    heart_cycle_predictions.append(output.item())

predicted_EF = sum(heart_cycle_predictions)/len(heart_cycle_predictions)

print("Predicted EF: " + str(predicted_EF))